# Knowledge Graph Lab with Neo4j and Ollama

In [ ]:
# Install required packages
! pip install -qU langchain langchain-experimental langchain-openai langchain-neo4j neo4j wikipedia

In [ ]:
# Install langchain-experimental for graph transformers
! pip install -qU langchain-experimental

In [ ]:
import os
from langchain_community.document_loaders import WikipediaLoader
from langchain_text_splitters import TokenTextSplitter
from langchain_experimental.graph_transformers.diffbot import DiffbotGraphTransformer
from langchain_community.graphs import Neo4jGraph

# Load Wikipedia data about Satoshi Nakamoto
print("Loading Wikipedia article...")
raw_documents = WikipediaLoader(query="Satoshi Nakamoto").load()

# Split documents into smaller chunks
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents)
print(f"Loaded {len(documents)} document chunks")

Loading Wikipedia article...
Loaded 48 document chunks


In [ ]:
# Extract entities and relationships using Diffbot
print("Extracting entities and relationships...")

diffbot_api_key = "ff5d2ce9684be173cd0e18b29b6270ed"
diffbot_nlp = DiffbotGraphTransformer(diffbot_api_key=diffbot_api_key)

# Convert documents to graph format
graph_documents = diffbot_nlp.convert_to_graph_documents(documents)

print(f"Extracted {len(graph_documents)} graph documents")
print(f"Total nodes: {sum(len(doc.nodes) for doc in graph_documents)}")
print(f"Total relationships: {sum(len(doc.relationships) for doc in graph_documents)}")

Extracting entities and relationships...
Extracted 48 graph documents
Total nodes: 249
Total relationships: 313


In [ ]:
# Connection details for our local Docker container
DB_URL = "bolt://localhost:7687"
DB_AUTH = ("neo4j", "password")

print("🔌 Establishing connection to Neo4j...")

try:
    # Initialize the driver
    graph = Neo4jGraph(
        url=DB_URL,
        username=DB_AUTH[0],
        password=DB_AUTH[1]
    )

    # 1. RESET: Clear any old data to ensure a clean lab environment
    print("🧹 Clearing existing database data...")
    graph.query("MATCH (n) DETACH DELETE n")

    # 2. LOAD: Push the extracted graph documents into Neo4j
    print("🌱 Populating the Knowledge Graph...")
    graph.add_graph_documents(graph_documents)
    
    print("✅ Database is live and populated!")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("Check if your 'mini_lab_neo4j' container is running.")

🔌 Establishing connection to Neo4j...
🧹 Clearing existing database data...
🌱 Populating the Knowledge Graph...
✅ Database is live and populated!


In [ ]:
# Check what node labels exist in the database
print("Node labels in the database:")
labels = graph.query("CALL db.labels()")
for label in labels:
    print(f"  - {label}")

Node labels in the database:
  - {'label': 'Person'}
  - {'label': 'Skill'}
  - {'label': 'Location'}
  - {'label': 'Organization'}
  - {'label': 'Award'}


In [ ]:
# Look at some sample relationships
print("Sample relationships:")
results = graph.query("""
MATCH (a)-[r]->(b)
RETURN labels(a)[0] as source_type, 
       a.id as source, 
       type(r) as relationship, 
       labels(b)[0] as target_type, 
       b.id as target
LIMIT 5
""")
for r in results:
    print(f"  {r['source_type']}({r['source']}) --[{r['relationship']}]--> {r['target_type']}({r['target']})")

Sample relationships:
  Person(http://www.wikidata.org/entity/Q19600251) --[EDUCATED_AT]--> Organization(http://www.wikidata.org/entity/Q41506)
  Person(http://www.wikidata.org/entity/Q19600251) --[POLITICAL_AFFILIATION]--> Organization(http://www.wikidata.org/entity/Q558334)
  Person(http://www.wikidata.org/entity/Q19600251) --[EMPLOYEE_OR_MEMBER_OF]--> Organization(http://www.wikidata.org/entity/Q558334)
  Person(http://www.wikidata.org/entity/Q19600251) --[EMPLOYEE_OR_MEMBER_OF]--> Organization(MemoryDealers.com)
  Person(http://www.wikidata.org/entity/Q19600251) --[PERSON_LOCATION]--> Location(http://www.wikidata.org/entity/Q781)


In [ ]:
# Find Satoshi Nakamoto in the graph
print("Looking for Satoshi Nakamoto...")
results = graph.query("""
MATCH (p:Person)
WHERE p.id CONTAINS 'Satoshi Nakamoto' OR p.name CONTAINS 'Satoshi Nakamoto'
RETURN p.id as id, p.name as name
LIMIT 5
""")
for result in results:
    print(f"  Found: {result['name']} (ID: {result['id']})")

Looking for Satoshi Nakamoto...
  Found: Satoshi Nakamoto (ID: http://www.wikidata.org/entity/Q13382352)


In [ ]:
# Set up the LLM for Graph-RAG
from langchain_community.chat_models import ChatOllama
from langchain_neo4j import Neo4jGraph, GraphCypherQAChain
from langchain_core.prompts import PromptTemplate

# Use Ollama with llama3
llm = ChatOllama(
    model="llama3",
    temperature=0
)

# Create a chain that converts questions to Cypher queries
graph_qa_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    return_intermediate_steps=True,
    allow_dangerous_requests=True,
    cypher_prompt=PromptTemplate(
        input_variables=["schema", "question"],
        template="""
You are a Neo4j Cypher query expert. Write a Cypher query to answer the question.

Schema:
{schema}

Question: {question}

Tips:
- Use MATCH to find nodes
- Use WHERE to filter
- Use RETURN to specify output
- Use LIMIT to restrict results

Cypher Query:
"""
    )
)

print("Graph-RAG chain ready!")

Graph-RAG chain ready!


C:\Users\arnau\AppData\Local\Temp\ipykernel_27776\1599148529.py:7: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(


In [ ]:
# Ask questions about the graph
question = "What organizations did Satoshi Nakamoto work with?"
print(f"Question: {question}")
print("="*50)

result = graph_qa_chain.invoke({"query": question})
print(f"\nAnswer: {result['result']}")

Question: What organizations did Satoshi Nakamoto work with?


> Entering new GraphCypherQAChain chain...
Generated Cypher:

MATCH (p:Person {name: "Satoshi Nakamoto"})-[:EMPLOYEE_OR_MEMBER_OF]->(o:Organization)
RETURN o.name AS organization_name
LIMIT 100;

Full Context:
[{'organization_name': 'Metzdowd Cryptography'}]

> Finished chain.

Answer: Metzdowd Cryptography.


In [ ]:
# Ask another question
question = "Who are the most connected people in the Bitcoin ecosystem?"
print(f"Question: {question}")
print("="*50)

result = graph_qa_chain.invoke({"query": question})
print(f"\nAnswer: {result['result']}")

Question: Who are the most connected people in the Bitcoin ecosystem?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person)-[r]-(o)
WHERE o.name CONTAINS 'Bitcoin' OR o.name CONTAINS 'Blockchain'
WITH p, COUNT(r) AS degree
ORDER BY degree DESC
LIMIT 10
RETURN p.name AS name, degree AS degree;

Full Context:
[{'name': 'Gavin Andresen', 'degree': 2}, {'name': 'Craig Steven Wright', 'degree': 2}, {'name': 'Roger Ver', 'degree': 1}]

> Finished chain.

Answer: Gavin Andresen, Craig Steven Wright, and Roger Ver are the most connected people in the Bitcoin ecosystem.


In [ ]:
# Helper function to ask questions
def ask(question):
    """Ask a question and get an answer from the knowledge graph."""
    try:
        result = graph_qa_chain.invoke({"query": question})
        return result['result']
    except Exception as e:
        return f"Error: {e}"

# Try it out
answer = ask("Who are Satoshi Nakamoto's collaborators?")
print(answer)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (satoshi:Person {name: "Satoshi Nakamoto"})-[:WORK_RELATIONSHIP]->(collaborator:Person)
RETURN collaborator.name AS CollaboratorName
LIMIT 10;

Full Context:
[{'CollaboratorName': 'W. Scott Stornetta'}, {'CollaboratorName': 'Mike Hearn'}]

> Finished chain.
W. Scott Stornetta, Mike Hearn are Satoshi Nakamoto's collaborators.
